In [1]:
import networkx as nx
import scipy as sp
import numpy as np
from scipy.optimize import fsolve 
import math

In [ ]:
n = 85
p = 0.3
G = nx.gnp_random_graph(n, p, None, True)
A = nx.adjacency_matrix(G)
np.set_printoptions(threshold=np.nan)


In [2]:
data = np.genfromtxt('1950wtw.dat', dtype=[('a','|S5'),('b','|S5'),('amount','f8')], skip_header=1, usemask=True)

In [3]:
def create_edgelist_binary():
    el = []
    for i in range(len(data)):
        el.append([data[i][0].decode('UTF-8'), data[i][1].decode('UTF-8')])
    return(el)

binary_edgelist = create_edgelist_binary()

In [6]:
H = nx.MultiDiGraph()
H.add_edges_from(binary_edgelist)
B = nx.adjacency_matrix(H)
np.set_printoptions(threshold=np.nan)
print(B)
n = len(H.nodes())
H_nodes = np.asarray(H.nodes())

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (0, 15)	1
  (0, 16)	1
  (0, 17)	1
  (0, 18)	1
  (0, 19)	1
  (0, 20)	1
  (0, 21)	1
  (0, 22)	1
  (0, 23)	1
  (0, 24)	1
  (0, 25)	1
  :	:
  (75, 70)	1
  (76, 13)	1
  (76, 32)	1
  (76, 35)	1
  (76, 60)	1
  (76, 77)	1
  (77, 16)	1
  (77, 69)	1
  (77, 76)	1
  (78, 26)	1
  (78, 56)	1
  (78, 60)	1
  (79, 60)	1
  (80, 60)	1
  (81, 60)	1
  (82, 32)	1
  (82, 40)	1
  (82, 41)	1
  (82, 49)	1
  (82, 60)	1
  (82, 69)	1
  (82, 74)	1
  (83, 60)	1
  (84, 59)	1
  (84, 61)	1


In [7]:
def full_dyad(M,n):
    fulldyad = 0
    for i in range (n):
        for j in range(n):
            if i == j:
                fulldyad += 0
            else:
                fulldyad += (M[i,j]*M[j,i])
    return fulldyad

def single_dyad(M,n):
    sindyad = 0
    for i in range (n):
        for j in range(n):
            if i == j:
                sindyad += 0
            else:
                sindyad += (M[i,j]*(1-M[j,i]))
    return sindyad

def empty_dyad(M,n):
    empdyad = 0
    for i in range (n):
        for j in range(n):
            if i == j:
                empdyad += 0
            else:
                empdyad += ((1-M[i,j])*(1-M[j,i]))
    return empdyad

def tri_one(M,n):
    one = 0
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if i == j or i == k or j == k:
                    one += 0
                else:
                    one += (1-M[i,j])*M[j,i]*M[j,k]*(1-M[k,j])*(1-M[i,k])*(1-M[k,i])
    return one

def tri_two(M,n):
    two = 0
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if i == j or i == k or j == k:
                    two += 0
                else:
                    two += M[i,j]*(1-M[j,i])*M[j,k]*(1-M[k,j])*(1-M[i,k])*(1-M[k,i])
    return two

def tri_three(M,n):
    three = 0
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if i == j or i == k or j == k:
                    three += 0
                else:
                    three += M[i,j]*(M[j,i])*M[j,k]*(1-M[k,j])*(1-M[i,k])*(1-M[k,i])
    return three

def tri_four(M,n):
    four = 0
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if i == j or i == k or j == k:
                    four += 0
                else:
                    four += (1-M[i,j])*(1-M[j,i])*M[j,k]*(1-M[k,j])*(M[i,k])*(1-M[k,i])
    return four

def tri_five(M,n):
    five = 0
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if i == j or i == k or j == k:
                    five += 0
                else:
                    five += (1-M[i,j])*M[j,i]*M[j,k]*(1-M[k,j])*(M[i,k])*(1-M[k,i])
    return five

def tri_six(M,n):
    six = 0
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if i == j or i == k or j == k:
                    six += 0
                else:
                    six += M[i,j]*M[j,i]*M[j,k]*(1-M[k,j])*M[i,k]*(1-M[k,i])
    return six

def tri_seven(M,n):
    seven = 0
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if i == j or i == k or j == k:
                    seven += 0
                else:
                    seven += M[i,j]*M[j,i]*(1-M[j,k])*M[k,j]*(1-M[i,k])*(1-M[k,i])
    return seven

def tri_eight(M,n):
    eight = 0
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if i == j or i == k or j == k:
                    eight += 0
                else:
                    eight += M[i,j]*M[j,i]*M[j,k]*M[k,j]*(1-M[i,k])*(1-M[k,i])
    return eight

def tri_nine(M,n):
    nine = 0
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if i == j or i == k or j == k:
                    nine += 0
                else:
                    nine += (1-M[i,j])*M[j,i]*(1-M[j,k])*M[k,j]*M[i,k]*(1-M[k,i])
    return nine

def tri_ten(M,n):
    ten = 0
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if i == j or i == k or j == k:
                    ten += 0
                else:
                    ten += (1-M[i,j])*M[j,i]*M[j,k]*M[k,j]*M[i,k]*(1-M[k,i])
    return ten

def tri_eleven(M,n):
    eleven = 0
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if i == j or i == k or j == k:
                    eleven += 0
                else:
                    eleven += M[i,j]*(1-M[j,i])*M[j,k]*M[k,j]*M[i,k]*(1-M[k,i])
    return eleven

def tri_twelve(M,n):
    twelve = 0
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if i == j or i == k or j == k:
                    twelve += 0
                else:
                    twelve += M[i,j]*M[j,i]*M[j,k]*M[k,j]*M[i,k]*(1-M[k,i])
    return twelve

def tri_thirteen(M,n):
    thirteen = 0
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if i == j or i == k or j == k:
                    thirteen += 0
                else:
                    thirteen += M[i,j]*M[j,i]*M[j,k]*M[k,j]*M[i,k]*M[k,i]
    return thirteen

print(tri_one(B.todense(),85))
print(tri_thirteen(B.todense(),85))

1484
58488


NULL B: Now we will create a function that will generate a list of all $2N$ coupled equations. First we create a vector x and vector y and then make that into a function or something. 

In [8]:
def f(z):
    F = [0]*2*n
    for i in range(n): 
        F[i] = -H.out_degree(H_nodes[i])
        for j in range(n):
            if i == j:
                None
            else:
                F[i] = F[i] + (z[i]*z[n+j])/(1+z[i]*z[n+j]) # z[i] = x[i] and z[n+j] = y[j]
        F[n+i] = -H.in_degree(H_nodes[i])
        for k in range(n):
            if i == k:
                None
            else:
                F[n+i] = F[n+i] + (z[n+i]*z[k])/(1 + z[n+i]*z[k]) # z[n+i] = y[i] and z[k] = x[k]
    return(F)

In [10]:
w = 2*n*[0.5]
r = fsolve(f, w)

In [11]:
print(r)

[2.44486970e+01 5.09385537e+00 7.28980876e-01 1.33921776e-01
 1.65485376e-01 1.02125017e+00 1.82636556e-01 1.19860192e-01
 1.82916990e-01 1.07527921e-01 1.65302059e-01 3.50646893e-01
 5.47234008e-01 1.09735320e+00 1.65690894e-01 4.71379514e-01
 6.19829236e-01 1.33866451e-01 1.65485376e-01 5.58336148e-01
 1.11853259e+00 9.28396322e-01 1.46175755e+01 1.64923775e+00
 2.03910723e+01 1.25354184e+01 7.26221108e+00 1.08326343e+01
 5.09504092e+00 8.69933354e-01 4.27815077e-01 7.26163985e+00
 1.12672262e+00 1.12193498e+00 7.25647719e-01 1.13110735e+00
 8.25003293e+00 5.10637163e-01 7.13868220e-01 2.46890452e-01
 1.50558916e-01 4.67539216e-01 1.02638009e+00 4.56304064e+00
 2.07181810e+00 1.89121095e+00 1.49345980e-01 6.18376053e-03
 9.55600938e-02 1.11471955e+00 3.98230670e-01 6.57921245e-01
 4.65375650e-01 1.71935931e+00 1.69379461e+00 1.38320175e+00
 1.39691508e+00 1.84455598e-01 6.18357975e-03 2.76828850e-01
 2.03941383e+01 7.47643926e-02 1.45577129e+00 1.12909258e+00
 6.14193832e-01 2.229850

now that I have the values for $x_i$ and $y_i$ I can calculate the probability matrix. $p_{ij} = \frac{x_i y_j}{1 + x_i y_j}$. This will be a 30 $\times$ 30 matrix.

In [12]:
def calc_p_M(r):
    M_p = []
    m = int(len(r)/2)
    for i in range(m):
        M_p.append([])
        for j in range(m,2*m):
            M_p[i].append((r[i]*r[j])/(1+r[i]*r[j]))
    return(M_p)

M_p = np.array(calc_p_M(r))

We now have an $N$ x $N$ matrix where every entry is the probability of a link between two nodes $i$ and $j$. We can now use these to calculate the expected value of certain motifs $\langle X \rangle$. 

In [14]:
tri_values = [[],
          []]
tri_values[0].append(tri_one(B.todense(),n))
tri_values[0].append(tri_two(B.todense(),n))
tri_values[0].append(tri_three(B.todense(),n))
tri_values[0].append(tri_four(B.todense(),n))
tri_values[0].append(tri_five(B.todense(),n))
tri_values[0].append(tri_six(B.todense(),n))
tri_values[0].append(tri_seven(B.todense(),n))
tri_values[0].append(tri_eight(B.todense(),n))
tri_values[0].append(tri_nine(B.todense(),n))
tri_values[0].append(tri_ten(B.todense(),n))
tri_values[0].append(tri_eleven(B.todense(),n))
tri_values[0].append(tri_twelve(B.todense(),n))
tri_values[0].append(tri_thirteen(B.todense(),n))

tri_values[1].append(int(tri_one(M_p,n)))
tri_values[1].append(int(tri_two(M_p,n)))
tri_values[1].append(int(tri_three(M_p,n)))
tri_values[1].append(int(tri_four(M_p,n)))
tri_values[1].append(int(tri_five(M_p,n)))
tri_values[1].append(int(tri_six(M_p,n)))
tri_values[1].append(int(tri_seven(M_p,n)))
tri_values[1].append(int(tri_eight(M_p,n)))
tri_values[1].append(int(tri_nine(M_p,n)))
tri_values[1].append(int(tri_ten(M_p,n)))
tri_values[1].append(int(tri_eleven(M_p,n)))
tri_values[1].append(int(tri_twelve(M_p,n)))
tri_values[1].append(int(tri_thirteen(M_p,n)))



di_values = [[],[]]

di_values[0].append(single_dyad(B.todense(),n))
di_values[0].append(full_dyad(B.todense(),n))
di_values[0].append(empty_dyad(B.todense(),n))

di_values[1].append(single_dyad(M_p,n))
di_values[1].append(full_dyad(M_p,n))
di_values[1].append(empty_dyad(M_p,n))

print("triads")
print(tri_values[0])
print(tri_values[1])
print("dyads")
print(di_values)

triads
[1484, 933, 2810, 1900, 542, 1162, 5560, 21784, 135, 783, 2768, 7402, 58488]
[3342, 3026, 4472, 4707, 1894, 2492, 6537, 12772, 1074, 2580, 4518, 9794, 42807]
dyads
[[485, 2386, 3784], [859.3292305188269, 2011.6707701270832, 3409.6707688352562]]


Now that we have both the predicted values and the actual values for the dyads and triads, we now continue to calculate the standard deviation $\sigma$. First we need to calculate the $\sigma^*[a_{ij}]$

In [15]:
def calc_std_aij(v):
    M_std = []
    k = int(len(v)/2)
    for i in range(k):
        M_std.append([])
        for j in range(k,2*k):
            M_std[i].append((math.sqrt(v[i]*v[j]))/(1 + v[i]*v[j]))
    return(M_std)

M_std = np.array(calc_std_aij(r))

Below we calculate the $\sigma[X]$ of the diads, which formally is the $\sigma[N_x]$

In [17]:
def calc_std_sing(K, M, n):
    std_sing = 0
    for e in range(n):
        for f in range(n):
            deriv = 0
            for i in range(n):
                for j in range(n):
                        if i == j:
                            deriv += 0
                        else:
                            if e == i and f == j:
                                deriv += (1-M[j,i])
                            elif e == j and f == i:
                                deriv += -M[i,j]
                            else:
                                deriv += 0
            std_sing += (K[e,f]*deriv)**2
    std_sing = math.sqrt(std_sing)
    return std_sing

def calc_std_full(K, M, n):
    std_full = 0
    for e in range(n):
        for f in range(n):
            deriv = 0
            for i in range(n):
                for j in range(n):
                        if i == j:
                            deriv += 0
                        else:
                            if e == i and f == j:
                                deriv += M[j,i]
                            elif e == j and f == i:
                                deriv += M[i,j]
                            else:
                                deriv += 0
            std_full += (K[e,f]*deriv)**2
    std_full = math.sqrt(std_full)
    return std_full

def calc_std_emp(K, M, n):
    std_emp = 0
    for e in range(n):
        for f in range(n):
            deriv = 0
            for i in range(n):
                for j in range(n):
                        if i == j:
                            deriv += 0
                        else:
                            if e == i and f == j:
                                deriv += -(1-M[j,i])
                            elif e == j and f == i:
                                deriv += -(1-M[i,j])
                            else:
                                deriv += 0
            std_emp += (K[e,f]*deriv)**2
    std_emp = math.sqrt(std_emp)
    return std_emp

std_di = []

std_di.append(calc_std_sing(M_std, M_p, 85))
std_di.append(calc_std_full(M_std, M_p, 85))
std_di.append(calc_std_emp(M_std, M_p, 85))

print(std_di)

[15.968912132329898, 30.712820629604277, 33.2741883695764]


Now the Z-scores of the diads.

In [18]:
def z_di_scores():
    z_di_scores = []
    for i in range(3):
        z = (di_values[0][i] - di_values[1][i])/std_di[i]
        z_di_scores.append(z)
    return z_di_scores

z_di_scores()

[-23.441122815183995, 12.188044673177902, 11.249838072895095]

Below we calculate the $\sigma[X]$ of the triads, which formally is the $\sigma[N_y]$

In [19]:
def calc_std_1(K, M, n):
    std_1 = 0
    for e in range(n):
        for f in range(n):
            deriv = 0
            for i in range(n):
                for j in range(n):
                    for k in range(n):
                        if i == j or i == k or j == k:
                            deriv += 0
                        else:
                            if e == i and f == j:
                                deriv += -M[j,i]*M[j,k]*(1-M[k,j])*(1-M[i,k])*(1-M[k,i])
                            elif e == j and f == i:
                                deriv += (1-M[i,j])*M[j,k]*(1-M[k,j])*(1-M[i,k])*(1-M[k,i])
                            elif e == j and f == k:
                                deriv += (1-M[i,j])*M[j,i]*(1-M[k,j])*(1-M[i,k])*(1-M[k,i])
                            elif e == k and f == j:
                                deriv += -(1-M[i,j])*M[j,i]*M[j,k]*(1-M[i,k])*(1-M[k,i])
                            elif e == i and f == k:
                                deriv += -(1-M[i,j])*M[j,i]*M[j,k]*(1-M[k,j])*(1-M[k,i])
                            elif e == k and f == i:
                                deriv += -(1-M[i,j])*M[j,i]*M[j,k]*(1-M[k,j])*(1-M[i,k])
                            else:
                                deriv += 0
            std_1 += (K[e,f]*deriv)**2
    std_1 = math.sqrt(std_1)
    return std_1

def calc_std_2(K, M, n):
    std_2 = 0
    for e in range(n):
        for f in range(n):
            deriv = 0
            for i in range(n):
                for j in range(n):
                    for k in range(n):
                        if i == j or i == k or j == k:
                            deriv += 0
                        else:
                            if e == i and f == j:
                                deriv += (1-M[j,i])*M[j,k]*(1-M[k,j])*(1-M[i,k])*(1-M[k,i])
                            elif e == j and f == i:
                                deriv += -M[i,j]*M[j,k]*(1-M[k,j])*(1-M[i,k])*(1-M[k,i])
                            elif e == j and f == k:
                                deriv += M[i,j]*(1-M[j,i])*(1-M[k,j])*(1-M[i,k])*(1-M[k,i])
                            elif e == k and f == j:
                                deriv += -M[i,j]*(1-M[j,i])*M[j,k]*(1-M[i,k])*(1-M[k,i])
                            elif e == i and f == k:
                                deriv += -M[i,j]*(1-M[j,i])*M[j,k]*(1-M[k,j])*(1-M[k,i])
                            elif e == k and f == i:
                                deriv += -M[i,j]*(1-M[j,i])*M[j,k]*(1-M[k,j])*(1-M[i,k])
                            else:
                                deriv += 0
            std_2 += (K[e,f]*deriv)**2
    std_2 = math.sqrt(std_2)
    return std_2

def calc_std_3(K, M, n):
    std_3 = 0
    for e in range(n):
        for f in range(n):
            deriv = 0
            for i in range(n):
                for j in range(n):
                    for k in range(n):
                        if i == j or i == k or j == k:
                            deriv += 0
                        else:
                            if e == i and f == j:
                                deriv += M[j,i]*M[j,k]*(1-M[k,j])*(1-M[i,k])*(1-M[k,i])
                            elif e == j and f == i:
                                deriv += M[i,j]*M[j,k]*(1-M[k,j])*(1-M[i,k])*(1-M[k,i])
                            elif e == j and f == k:
                                deriv += M[i,j]*M[j,i]*(1-M[k,j])*(1-M[i,k])*(1-M[k,i])
                            elif e == k and f == j:
                                deriv += -M[i,j]*M[j,i]*M[j,k]*(1-M[i,k])*(1-M[k,i])
                            elif e == i and f == k:
                                deriv += -M[i,j]*M[j,i]*M[j,k]*(1-M[k,j])*(1-M[k,i])
                            elif e == k and f == i:
                                deriv += -M[i,j]*M[j,i]*M[j,k]*(1-M[k,j])*(1-M[i,k])
                            else:
                                deriv += 0
            std_3 += (K[e,f]*deriv)**2
    std_3 = math.sqrt(std_3)
    return std_3

def calc_std_4(K, M, n):
    std_4 = 0
    for e in range(n):
        for f in range(n):
            deriv = 0
            for i in range(n):
                for j in range(n):
                    for k in range(n):
                        if i == j or i == k or j == k:
                            deriv += 0
                        else:
                            if e == i and f == j:
                                deriv += -(1-M[j,i])*M[j,k]*(1-M[k,j])*M[i,k]*(1-M[k,i])
                            elif e == j and f == i:
                                deriv += -(1-M[i,j])*M[j,k]*(1-M[k,j])*M[i,k]*(1-M[k,i])
                            elif e == j and f == k:
                                deriv += (1-M[i,j])*(1-M[j,i])*(1-M[k,j])*M[i,k]*(1-M[k,i])
                            elif e == k and f == j:
                                deriv += -(1-M[i,j])*(1-M[j,i])*M[j,k]*M[i,k]*(1-M[k,i])
                            elif e == i and f == k:
                                deriv += (1-M[i,j])*(1-M[j,i])*M[j,k]*(1-M[k,j])*(1-M[k,i])
                            elif e == k and f == i:
                                deriv += (1-M[i,j])*(1-M[j,i])*M[j,k]*(1-M[k,j])*M[i,k]
                            else:
                                deriv += 0
            std_4 += (K[e,f]*deriv)**2
    std_4 = math.sqrt(std_4)
    return std_4

def calc_std_5(K, M, n):
    std_5 = 0
    for e in range(n):
        for f in range(n):
            deriv = 0
            for i in range(n):
                for j in range(n):
                    for k in range(n):
                        if i == j or i == k or j == k:
                            deriv += 0
                        else:
                            if e == i and f == j:
                                deriv += -M[j,i]*M[j,k]*(1-M[k,j])*M[i,k]*(1-M[k,i])
                            elif e == j and f == i:
                                deriv += (1-M[i,j])*M[j,k]*(1-M[k,j])*M[i,k]*(1-M[k,i])
                            elif e == j and f == k:
                                deriv += (1-M[i,j])*M[j,i]*(1-M[k,j])*M[i,k]*(1-M[k,i])
                            elif e == k and f == j:
                                deriv += -(1-M[i,j])*M[j,i]*M[j,k]*M[i,k]*(1-M[k,i])
                            elif e == i and f == k:
                                deriv += (1-M[i,j])*M[j,i]*M[j,k]*(1-M[k,j])*(1-M[k,i])
                            elif e == k and f == i:
                                deriv += -(1-M[i,j])*M[j,i]*M[j,k]*(1-M[k,j])*M[i,k]
                            else:
                                deriv += 0
            std_5 += (K[e,f]*deriv)**2
    std_5 = math.sqrt(std_5)
    return std_5

def calc_std_6(K, M, n):
    std_6 = 0
    for e in range(n):
        for f in range(n):
            deriv = 0
            for i in range(n):
                for j in range(n):
                    for k in range(n):
                        if i == j or i == k or j == k:
                            deriv += 0
                        else:
                            if e == i and f == j:
                                deriv += M[j,i]*M[j,k]*(1-M[k,j])*M[i,k]*(1-M[k,i])
                            elif e == j and f == i:
                                deriv += M[i,j]*M[j,k]*(1-M[k,j])*M[i,k]*(1-M[k,i])
                            elif e == j and f == k:
                                deriv += M[i,j]*M[j,i]*(1-M[k,j])*M[i,k]*(1-M[k,i])
                            elif e == k and f == j:
                                deriv += -M[i,j]*M[j,i]*M[j,k]*M[i,k]*(1-M[k,i])
                            elif e == i and f == k:
                                deriv += M[i,j]*M[j,i]*M[j,k]*(1-M[k,j])*(1-M[k,i])
                            elif e == k and f == i:
                                deriv += -M[i,j]*M[j,i]*M[j,k]*(1-M[k,j])*M[i,k]
                            else:
                                deriv += 0
            std_6 += (K[e,f]*deriv)**2
    std_6 = math.sqrt(std_6)
    return std_6

def calc_std_7(K, M, n):
    std_7 = 0
    for e in range(n):
        for f in range(n):
            deriv = 0
            for i in range(n):
                for j in range(n):
                    for k in range(n):
                        if i == j or i == k or j == k:
                            deriv += 0
                        else:
                            if e == i and f == j:
                                deriv += M[j,i]*(1-M[j,k])*M[k,j]*(1-M[i,k])*(1-M[k,i])
                            elif e == j and f == i:
                                deriv += M[i,j]*(1-M[j,k])*M[k,j]*(1-M[i,k])*(1-M[k,i])
                            elif e == j and f == k:
                                deriv += -M[i,j]*M[j,i]*M[k,j]*(1-M[i,k])*(1-M[k,i])
                            elif e == k and f == j:
                                deriv += M[i,j]*M[j,i]*(1-M[j,k])*(1-M[i,k])*(1-M[k,i])
                            elif e == i and f == k:
                                deriv += -M[i,j]*M[j,i]*(1-M[j,k])*M[k,j]*(1-M[k,i])
                            elif e == k and f == i:
                                deriv += -M[i,j]*M[j,i]*(1-M[j,k])*M[k,j]*(1-M[i,k])
                            else:
                                deriv += 0
            std_7 += (K[e,f]*deriv)**2
    std_7 = math.sqrt(std_7)
    return std_7

def calc_std_8(K, M, n):
    std_8 = 0
    for e in range(n):
        for f in range(n):
            deriv = 0
            for i in range(n):
                for j in range(n):
                    for k in range(n):
                        if i == j or i == k or j == k:
                            deriv += 0
                        else:
                            if e == i and f == j:
                                deriv += M[j,i]*M[j,k]*M[k,j]*(1-M[i,k])*(1-M[k,i])
                            elif e == j and f == i:
                                deriv += M[i,j]*M[j,k]*M[k,j]*(1-M[i,k])*(1-M[k,i])
                            elif e == j and f == k:
                                deriv += M[i,j]*M[j,i]*M[k,j]*(1-M[i,k])*(1-M[k,i])
                            elif e == k and f == j:
                                deriv += M[i,j]*M[j,i]*M[j,k]*(1-M[i,k])*(1-M[k,i])
                            elif e == i and f == k:
                                deriv += -M[i,j]*M[j,i]*M[j,k]*M[k,j]*(1-M[k,i])
                            elif e == k and f == i:
                                deriv += -M[i,j]*M[j,i]*M[j,k]*M[k,j]*(1-M[i,k])
                            else:
                                deriv += 0
            std_8 += (K[e,f]*deriv)**2
    std_8 = math.sqrt(std_8)
    return std_8

def calc_std_9(K, M, n):
    std_9 = 0
    for e in range(n):
        for f in range(n):
            deriv = 0
            for i in range(n):
                for j in range(n):
                    for k in range(n):
                        if i == j or i == k or j == k:
                            deriv += 0
                        else:
                            if e == i and f == j:
                                deriv += -M[j,i]*(1-M[j,k])*M[k,j]*M[i,k]*(1-M[k,i])
                            elif e == j and f == i:
                                deriv += (1-M[i,j])*(1-M[j,k])*M[k,j]*M[i,k]*(1-M[k,i])
                            elif e == j and f == k:
                                deriv += -(1-M[i,j])*M[j,i]*M[k,j]*M[i,k]*(1-M[k,i])
                            elif e == k and f == j:
                                deriv += (1-M[i,j])*M[j,i]*(1-M[j,k])*M[i,k]*(1-M[k,i])
                            elif e == i and f == k:
                                deriv += (1-M[i,j])*M[j,i]*(1-M[j,k])*M[k,j]*(1-M[k,i])
                            elif e == k and f == i:
                                deriv += -(1-M[i,j])*M[j,i]*(1-M[j,k])*M[k,j]*M[i,k]
                            else:
                                deriv += 0
            std_9 += (K[e,f]*deriv)**2
    std_9 = math.sqrt(std_9)
    return std_9

def calc_std_10(K, M, n):
    std_10 = 0
    for e in range(n):
        for f in range(n):
            deriv = 0
            for i in range(n):
                for j in range(n):
                    for k in range(n):
                        if i == j or i == k or j == k:
                            deriv += 0
                        else:
                            if e == i and f == j:
                                deriv += -M[j,i]*M[j,k]*M[k,j]*M[i,k]*(1-M[k,i])
                            elif e == j and f == i:
                                deriv += (1-M[i,j])*M[j,k]*M[k,j]*M[i,k]*(1-M[k,i])
                            elif e == j and f == k:
                                deriv += (1-M[i,j])*M[j,i]*M[k,j]*M[i,k]*(1-M[k,i])
                            elif e == k and f == j:
                                deriv += (1-M[i,j])*M[j,i]*M[j,k]*M[i,k]*(1-M[k,i])
                            elif e == i and f == k:
                                deriv += (1-M[i,j])*M[j,i]*M[j,k]*M[k,j]*(1-M[k,i])
                            elif e == k and f == i:
                                deriv += -(1-M[i,j])*M[j,i]*M[j,k]*M[k,j]*M[i,k]
                            else:
                                deriv += 0
            std_10 += (K[e,f]*deriv)**2
    std_10 = math.sqrt(std_10)
    return std_10

def calc_std_11(K, M, n):
    std_11 = 0
    for e in range(n):
        for f in range(n):
            deriv = 0
            for i in range(n):
                for j in range(n):
                    for k in range(n):
                        if i == j or i == k or j == k:
                            deriv += 0
                        else:
                            if e == i and f == j:
                                deriv += (1-M[j,i])*M[j,k]*M[k,j]*M[i,k]*(1-M[k,i])
                            elif e == j and f == i:
                                deriv += -M[i,j]*M[j,k]*M[k,j]*M[i,k]*(1-M[k,i])
                            elif e == j and f == k:
                                deriv += M[i,j]*(1-M[j,i])*M[k,j]*M[i,k]*(1-M[k,i])
                            elif e == k and f == j:
                                deriv += M[i,j]*(1-M[j,i])*M[j,k]*M[i,k]*(1-M[k,i])
                            elif e == i and f == k:
                                deriv += M[i,j]*(1-M[j,i])*M[j,k]*M[k,j]*(1-M[k,i])
                            elif e == k and f == i:
                                deriv += -M[i,j]*(1-M[j,i])*M[j,k]*M[k,j]*M[i,k]
                            else:
                                deriv += 0
            std_11 += (K[e,f]*deriv)**2
    std_11 = math.sqrt(std_11)
    return std_11

def calc_std_12(K ,M ,n):
    std_12 = 0
    for e in range(n):
        for f in range(n):
            deriv = 0
            for i in range(n):
                for j in range(n):
                    for k in range(n):
                        if i == j or i == k or j == k:
                            deriv += 0
                        else:
                            if e == i and f == j:
                                deriv += M[j,i]*M[j,k]*M[k,j]*M[i,k]*(1-M[k,i])
                            elif e == j and f == i:
                                deriv += M[i,j]*M[j,k]*M[k,j]*M[i,k]*(1-M[k,i])
                            elif e == j and f == k:
                                deriv += M[i,j]*M[j,i]*M[k,j]*M[i,k]*(1-M[k,i])
                            elif e == k and f == j:
                                deriv += M[i,j]*M[j,i]*M[j,k]*M[i,k]*(1-M[k,i])
                            elif e == i and f == k:
                                deriv += M[i,j]*M[j,i]*M[j,k]*M[k,j]*(1-M[k,i])
                            elif e == k and f == i:
                                deriv += -M[i,j]*M[j,i]*M[j,k]*M[k,j]*M[i,k]
                            else:
                                deriv += 0
            std_12 += (K[e,f]*deriv)**2
    std_12 = math.sqrt(std_12)
    return std_12

def calc_std_13(K, M, n):
    std_13 = 0
    for e in range(n):
        for f in range(n):
            deriv = 0
            for i in range(n):
                for j in range(n):
                    for k in range(n):
                        if i == j or i == k or j == k:
                            deriv += 0
                        else:
                            if e == i and f == j:
                                deriv += M[j,i]*M[j,k]*M[k,j]*M[i,k]*M[k,i]
                            elif e == j and f == i:
                                deriv += M[i,j]*M[j,k]*M[k,j]*M[i,k]*M[k,i]
                            elif e == j and f == k:
                                deriv += M[i,j]*M[j,i]*M[k,j]*M[i,k]*M[k,i]
                            elif e == k and f == j:
                                deriv += M[i,j]*M[j,i]*M[j,k]*M[i,k]*M[k,i]
                            elif e == i and f == k:
                                deriv += M[i,j]*M[j,i]*M[j,k]*M[k,j]*M[k,i]
                            elif e == k and f == i:
                                deriv += M[i,j]*M[j,i]*M[j,k]*M[k,j]*M[i,k]
                            else:
                                deriv += 0
            std_13 += (K[e,f]*deriv)**2
    std_13 = math.sqrt(std_13)
    return std_13

std_tri = []

std_tri.append(calc_std_1(M_std, M_p, 85))
std_tri.append(calc_std_2(M_std, M_p, 85))
std_tri.append(calc_std_3(M_std, M_p, 85))
std_tri.append(calc_std_4(M_std, M_p, 85))
std_tri.append(calc_std_5(M_std, M_p, 85))
std_tri.append(calc_std_6(M_std, M_p, 85))
std_tri.append(calc_std_7(M_std, M_p, 85))
std_tri.append(calc_std_8(M_std, M_p, 85))
std_tri.append(calc_std_9(M_std, M_p, 85))
std_tri.append(calc_std_10(M_std, M_p, 85))
std_tri.append(calc_std_11(M_std, M_p, 85))
std_tri.append(calc_std_12(M_std, M_p, 85))
std_tri.append(calc_std_13(M_std, M_p, 85))

print(std_tri)

[219.54489036389134, 107.91616069557196, 264.7551110289, 214.75611872957106, 112.78400958279273, 166.67374785551357, 296.88298283482476, 515.5772694048829, 64.81898793435455, 99.64271471339784, 279.78284306444795, 187.15225218868667, 1520.7686948071557]


In [20]:
def z_tri_scores():
    z_tri_scores = []
    for i in range(13):
        z = (tri_values[0][i] - tri_values[1][i])/std_tri[i]
        z_tri_scores.append(z)
    return z_tri_scores

z_tri_scores()

[-8.462961706466507,
 -19.394685527261167,
 -6.277499208763453,
 -13.070640392484833,
 -11.987514941180743,
 -7.979660967082547,
 -3.290858878710365,
 17.479436225732588,
 -14.486495854439635,
 -18.034434380563678,
 -6.254851015281476,
 -12.781037748818479,
 10.311232769023079]